In [17]:
import os
import pandas as pd
import numpy as np
import pymysql

import sqlalchemy as db
from sqlalchemy import (
    text,
    create_engine, 
    Float,
    Integer,
    String,
    Table,
    Date,
    MetaData,
    Table,
    Column
)

In [16]:
df = pd.read_csv('../origin_data/2019_1.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
dropped_col = [
    'DivAirportLandings', 
    'DivReachedDest', 
    'DivActualElapsedTime', 
    'DivArrDelay', 
    'DivDistance',
    'Div1Airport',
    'Div1AirportID',
    'Div1AirportSeqID',
    'Div1WheelsOn',
    'Div1TotalGTime',
    'Div1LongestGTime',
    'Div1WheelsOff',
    'Div1TailNum',
    'Div2Airport',
    'Div2AirportID',
    'Div2AirportSeqID',
    'Div2WheelsOn',
    'Div2TotalGTime',
    'Div2LongestGTime',
    'Div2WheelsOff',
    'Div2TailNum',
    'Div3Airport',
    'Div3AirportID',
    'Div3AirportSeqID',
    'Div3WheelsOn',
    'Div3TotalGTime',
    'Div3LongestGTime',
    'Div3WheelsOff',
    'Div3TailNum',
    'Div4Airport',
    'Div4AirportID',
    'Div4AirportSeqID',
    'Div4WheelsOn',
    'Div4TotalGTime',
    'Div4LongestGTime',
    'Div4WheelsOff',
    'Div4TailNum',
    'Div5Airport',
    'Div5AirportID',
    'Div5AirportSeqID',
    'Div5WheelsOn',
    'Div5TotalGTime',
    'Div5LongestGTime',
    'Div5WheelsOff',
    'Div5TailNum',
    'FirstDepTime',
    'TotalAddGTime',
    'LongestAddGTime'
]

df = df.drop(dropped_col, axis=1)
df['CarrierDelay'].fillna(0, inplace=True)
df['WeatherDelay'].fillna(0, inplace=True)
df['NASDelay'].fillna(0, inplace=True)
df['SecurityDelay'].fillna(0, inplace=True)
df['SecurityDelay'].fillna(0, inplace=True)
df['LateAircraftDelay'].fillna(0, inplace=True)


df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '../origin_data/2019_1.csv'

In [13]:
df.tail(10)

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
583975,2019,1,1,6,7,2019-01-06,HA,19690,HA,N490HA,...,51.0,37.0,1.0,216.0,1,0.0,0.0,0.0,0.0,0.0
583976,2019,1,1,2,3,2019-01-02,HA,19690,HA,N481HA,...,54.0,35.0,1.0,216.0,1,52.0,0.0,0.0,0.0,0.0
583977,2019,1,1,4,5,2019-01-04,HA,19690,HA,N494HA,...,59.0,41.0,1.0,216.0,1,0.0,0.0,0.0,0.0,0.0
583978,2019,1,1,6,7,2019-01-06,HA,19690,HA,N490HA,...,60.0,33.0,1.0,216.0,1,0.0,0.0,0.0,0.0,0.0
583979,2019,1,1,2,3,2019-01-02,HA,19690,HA,N483HA,...,42.0,21.0,1.0,102.0,1,0.0,0.0,0.0,0.0,0.0
583980,2019,1,1,2,3,2019-01-02,HA,19690,HA,N483HA,...,36.0,21.0,1.0,102.0,1,0.0,0.0,0.0,0.0,0.0
583981,2019,1,1,2,3,2019-01-02,HA,19690,HA,N481HA,...,42.0,24.0,1.0,100.0,1,0.0,0.0,0.0,0.0,0.0
583982,2019,1,1,2,3,2019-01-02,HA,19690,HA,N481HA,...,50.0,22.0,1.0,100.0,1,0.0,0.0,0.0,0.0,0.0
583983,2019,1,1,6,7,2019-01-06,HA,19690,HA,N487HA,...,55.0,34.0,1.0,163.0,1,0.0,0.0,0.0,0.0,0.0
583984,2019,1,1,6,7,2019-01-06,HA,19690,HA,N487HA,...,46.0,30.0,1.0,163.0,1,0.0,0.0,0.0,0.0,0.0


In [33]:
df.to_csv('test.csv', index=False)

In [4]:
hostname = "localhost"
db_name = "BTS"
user_name = "user"
password = "password"

engine = create_engine(
    f"mysql+pymysql://{user_name}:{password}@{hostname}:3306/{db_name}"
)
db_connection = engine.connect()
metadata = db.MetaData()

In [5]:
columns = list(df.columns)
dtypes = list(df.dtypes)
new_dtypes = []
nullable_flags = [True] * len(columns)

cols_n_dtypes = dict(zip(columns, dtypes))

for key in cols_n_dtypes:
    if str(cols_n_dtypes[key]) == 'int64':
        new_dtypes.append(Integer)
    elif str(cols_n_dtypes[key]) == 'object':
        if key == 'FlightDate':
            new_dtypes.append(Date)
        else:
            new_dtypes.append(String(200))
    elif str(cols_n_dtypes[key]) == 'float64':
        new_dtypes.append(Float)

In [6]:
Transportations = db.Table(
    'Transportations', 
    metadata,
    *(
        Column(
            column,
            new_dtype,
            nullable=nullable_flag
        )
        for column, new_dtype, nullable_flag in zip (columns, new_dtypes, nullable_flags)
    )
)

metadata.create_all(engine) 

In [10]:
df.to_sql('Transportations', db_connection, if_exists='append', index=False)

DataError: (pymysql.err.DataError) (1406, "Data too long for column 'DestCityName' at row 914")
[SQL: INSERT INTO `Transportations` (`Year`, `Quarter`, `Month`, `DayofMonth`, `DayOfWeek`, `FlightDate`, `Reporting_Airline`, `DOT_ID_Reporting_Airline`, `IATA_CODE_Reporting_Airline`, `Tail_Number`, `Flight_Number_Reporting_Airline`, `OriginAirportID`, `OriginAirportSeqID`, `OriginCityMarketID`, `Origin`, `OriginCityName`, `OriginState`, `OriginStateFips`, `OriginStateName`, `OriginWac`, `DestAirportID`, `DestAirportSeqID`, `DestCityMarketID`, `Dest`, `DestCityName`, `DestState`, `DestStateFips`, `DestStateName`, `DestWac`, `CRSDepTime`, `DepTime`, `DepDelay`, `DepDelayMinutes`, `DepDel15`, `DepartureDelayGroups`, `DepTimeBlk`, `TaxiOut`, `WheelsOff`, `WheelsOn`, `TaxiIn`, `CRSArrTime`, `ArrTime`, `ArrDelay`, `ArrDelayMinutes`, `ArrDel15`, `ArrivalDelayGroups`, `ArrTimeBlk`, `Cancelled`, `CancellationCode`, `Diverted`, `CRSElapsedTime`, `ActualElapsedTime`, `AirTime`, `Flights`, `Distance`